In [107]:
link = "https://www.reuters.com/resources/archive/us/{}{}{}.html"

In [108]:
from bs4 import BeautifulSoup as BS
import requests
from requests.exceptions import HTTPError
import re
import datetime
import pytz

In [109]:
r = requests.get(link.format("2017", "08", "09"))  # when does not load "2013", "08", "09"

In [110]:
soup = BS(r.content, 'html.parser')
span_list = soup.find_all('div' , attrs={'class':"headlineMed"})  #, 

In [111]:
k = str(span_list[0]).split('>')[1].split("=")[1].replace('"', '')
k

'http://www.reuters.com/article/us-mcdonalds-china-breakingviews/mcdonalds-boldly-supersizes-its-china-strategy-idUSKBN1AP0AI'

In [112]:
str(span_list[0]).split()[-2]

'11:59PM'

In [118]:
from newspaper import Article

In [119]:
article = Article(k.strip())
try:
    article.download()
except HTTPError:
    print('***FAILED TO DOWNLOAD***', article.url)


In [120]:
article.parse()

In [121]:
article.authors

['Alec Macfarlane', 'Min Read']

In [122]:
print(datetime.datetime.utcnow())
print(datetime.datetime.now())
UTC_OFFSET = datetime.datetime.now().hour - datetime.datetime.utcnow().hour
UTC_OFFSET

2018-10-19 09:44:04.282628
2018-10-19 12:44:04.282763


3

In [130]:
timezone = pytz.timezone('Etc/Greenwich')


print(article.publish_date.tzinfo)
print(article.publish_date)
#print(timezone.localize(article.publish_date))

print(article.publish_date - datetime.timedelta(hours=UTC_OFFSET))
# A v reuters pyshe sho published at 11:59 PM 

tzutc()
2017-08-09 03:59:52+00:00
2017-08-09 00:59:52+00:00


In [57]:
article.text


"HONG KONG (Reuters Breakingviews) - McDonald’s is going large in China. It will have to ensure that breakneck growth does not bring indigestion.\n\nCustomers play using their mobile phones at a McDonald's store in Beijing, China January 9, 2017. REUTERS/Jason Lee\n\nOn Tuesday, the $126 billion burger giant said it planned to open 2,000 mainland stores by 2022. That will nearly double its footprint from 2,500 outlets now. That target is up one-third from a previous goal of 1,500 restaurants in five years, a goal that also originally included Hong Kong.\n\nThis underlines how the $2.1 billion sale of McDonald’s China business, which was agreed in January and closed last week, was not really a retreat. It now holds one-fifth of the business, which has the rights to run McDonald’s operations in China for 20 years, with the rest owned by Chinese conglomerate CITIC, CITIC’s private equity arm, and U.S. buyout firm Carlyle.\n\nExpanding makes sense. Bigger rival Yum China, which operates Pi

In [58]:
article.title

'McDonald’s boldly supersizes its China strategy'